## Weights

### Option 1: 2 paremters per actor.

- Resistance $r$ - probability of keeping current opinion
- Charisma $p$ - probability of persuading the other person
- $\varepsilon$ - threshold for alternative option

Rules:
- $|r_A - p_B| \leq \varepsilon$ - Alice chooses option 3
- ~$ r_A - p_B > \varepsilon$ - Alice persuades Bob
- ~$ p_B - r_A > \varepsilon$ - Bob persuades Alice

In the dialog model, these parameters should blend in somehow.

For a given actor:
- high resistance, low persuasion - will stick to his option, but will not convince others.
- high resistance, high persuasion - will stick to his option, and will convince others.
- low resistance, low persuasion - will change his opinion, and will not convince others.
- low resistance, high persuasion - ready to change his own opinion, easily convinces others.

Feels like it's ok to stick to $p$ and $q$ for now.

**Pros:**
- Allows to better distinguish leaders

**Cons:**
- Resistance and persuasion are not independent.
- Complicates the model. It's fine if it's necessary, but is it?


### Option 2: 1 parameter per actor

- Resistance $r$ - probability of keeping current opinion. Higher resisance also means more chance others will listen

Rules:
- $|r_A - r_B| \leq \varepsilon$ - Alice chooses option 3
- $ r_A - r_B > \varepsilon$ - Alice persuades Bob
- $ r_B - r_A > \varepsilon$ - Bob persuades Alice

Given that in the dialog, we have a joint preference matrix: $w_A \cdot w_B$, how should we interpret the resistance?

**Cons**
- With one parameter, I don't see an option to allow e.g. only Alice going for option 3, while Bob keeping his opinion.

## Values with Option 1

(vA, vB)
- -> (vA, vA): Alice persuades Bob - Alice is charismatic AND Bob is not resistant $c_A \cdot (1 - r_B)$
- -> (vB, vB): Bob persuades Alice - Bob is charismatic AND Alice is not resistant $c_B \cdot (1 - r_A)$
- -> (vB, vA): They persuade each other - both are charismatic AND not resistant $c_A \cdot c_B \cdot (1 - r_A) \cdot (1 - r_B)$
- -> (vA, vB): Both keep their opinions - both are resistant and not charismatic $r_A \cdot r_B \cdot (1 - c_A) \cdot (1 - c_B)$

- -> (vA, vThird): Bob goes for third option - Bob is resistant AND Alice is charismatic $r_B \cdot c_A$
- -> (vB, vThird): Bob persuades Alice, but goes for third option himself **XXX DOESN'T MAKE SENSE**
- -> (vThird, vA): Alice persuades Bob, but goes for third option herself **XXX DOESN'T MAKE SENSE**
- -> (vThird, vB): Alice goes for third option - Alice is resistant AND Bob is charismatic $r_A \cdot c_B$
- -> (vThird, vThird): Both go for same option - both are charismatic AND resistant $c_A \cdot c_B \cdot r_A \cdot r_B$

What is the sum?
\begin{equation}
 c1(1 - r2) + c2(1 - r1) + c1 * c2 * (1 - r1) * (1 - r2) + r1 * r2 * (1 - c1) * (1 - c2) + r2 * c1 + r1 * c2 + c1 * c2 * r1 * r2
\end{equation}

In [1]:

def row_tostr(row):
    return "[" + ", ".join([f"{x:.2f}".rstrip('0').rstrip('.') for x in row]) + "]"


def sumup_and_print(c1, r1, c2, r2, title):
    print(f"{title}: c1: {c1}, r1: {r1}, c2: {c2}, r2: {r2}")
    va_va = c1 * (1 - r2)
    va_vb = r1 * r2 * (1 - c1) * (1 - c2)
    va_vthird = r2 * c1
    vb_va = c1 * c2 * (1 - r1) * (1 - r2)
    vb_vb = c2 * (1 - r1)
    vb_vthird = 0
    vthird_va = 0
    vthird_vb = r1 * c2
    vthird_vthird = c1 * c2 * r1 * r2

    row = [va_va, va_vb, va_vthird, vb_va, vb_vb, vb_vthird, vthird_va, vthird_vb, vthird_vthird]
    total = sum(row)
    normalized = [x / total for x in row]

    print(row_tostr(row), "=", total)
    print(row_tostr(normalized), "=", sum(normalized))

In [2]:
sumup_and_print(1, 0, 1, 0, "Top resistance, no charisma")

Top resistance, no charisma: c1: 1, r1: 0, c2: 1, r2: 0
[1, 0, 0, 1, 1, 0, 0, 0, 0] = 3
[0.33, 0, 0, 0.33, 0.33, 0, 0, 0, 0] = 1.0


In [3]:
sumup_and_print(1, 1, 1, 1, "Top resistance, top charisma")

Top resistance, top charisma: c1: 1, r1: 1, c2: 1, r2: 1
[0, 0, 1, 0, 0, 0, 0, 1, 1] = 3
[0, 0, 0.33, 0, 0, 0, 0, 0.33, 0.33] = 1.0


In [4]:
sumup_and_print(0, 1, 0, 1, "Top charisma, no resistance")

Top charisma, no resistance: c1: 0, r1: 1, c2: 0, r2: 1
[0, 1, 0, 0, 0, 0, 0, 0, 0] = 1
[0, 1, 0, 0, 0, 0, 0, 0, 0] = 1.0


In [5]:
sumup_and_print(0.8, 0.8, 0.8, 0.8, "High charisma, high resistance")

High charisma, high resistance: c1: 0.8, r1: 0.8, c2: 0.8, r2: 0.8
[0.16, 0.03, 0.64, 0.03, 0.16, 0, 0, 0.64, 0.41] = 2.0608000000000004
[0.08, 0.01, 0.31, 0.01, 0.08, 0, 0, 0.31, 0.2] = 0.9999999999999999


In [6]:
sumup_and_print(0.8, 0.8, 0.8, 0.2, "Bob not charismatic")


Bob not charismatic: c1: 0.8, r1: 0.8, c2: 0.8, r2: 0.2
[0.64, 0.01, 0.16, 0.1, 0.16, 0, 0, 0.64, 0.1] = 1.8112000000000004
[0.35, 0, 0.09, 0.06, 0.09, 0, 0, 0.35, 0.06] = 1.0
